In [1]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [2]:
from llm import llm
from graph import graph
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_session_id
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.physician import physician_cypher_qa
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 
from langchain_core.runnables import RunnableConfig  


@tool
def explore_hospital(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain"""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain."""
    return patient_cypher_qa.invoke(question)


tools = [explore_hospital, explore_patient]



system_message = SystemMessage(content="""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Always follow the required format strictly.
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

Use the following format to provide a response:
```
**Thought**
**Tool_used** (e.g. explore_patient)
**Response**
```

Begin!

Previous conversation history:
{{chat_history}}

New input: {{input}}
""")

memory = MemorySaver()

langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message, checkpointer=memory)

# Define the memory function
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Create the agent executor
agent_executor = RunnableWithMessageHistory(
    langgraph_agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)




/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_community/graphs/neo4j_graph.py:6: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.graphs.graph_document import GraphDocument


In [8]:
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler

def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and streams responses to be printed in the console.
    Returns the final output content from the last AIMessage after invocation.
    """
    # Define the messages structure using HumanMessage
    messages = [HumanMessage(content=user_input)]
    
    # Get the session ID and thread ID
    session_id = get_session_id()  # Assuming this generates a unique session ID
    thread_id = get_session_id()    # You can use the same function or create a new one for thread ID

    # Create a RunnableConfig instance
    config = RunnableConfig(
        configurable={
            "session_id": session_id,
            "thread_id": thread_id
        }
    )

    # Initialize the StreamlitCallbackHandler
    st_callback = StreamlitCallbackHandler(st.container())

    # Initialize a variable to hold the final output content
    final_output_content = ""

    # Prepare the input for the invoke method
    input_data = {
        "messages": messages,
        "input": user_input  # Ensure the input key is included
    }

    # Invoke the agent with the messages and configuration, including the callback
    response = agent_executor.invoke(input_data, config, callbacks=[st_callback])
    

    # Check if the response contains messages
    if 'messages' in response:
        for message in response['messages']:
            if isinstance(message, AIMessage):
                # Update the final output content with the latest AIMessage content
                final_output_content = message.content  # Store the latest AIMessage content


    # Return the final output content
    return final_output_content




In [14]:
query = "do Lam Wah Ee Hospital exist in the database? May I know its contact number"
    
messages = [HumanMessage(content=query)]
    
    # Get the session ID and thread ID
session_id = get_session_id()  # Assuming this generates a unique session ID
thread_id = get_session_id()    # You can use the same function or create a new one for thread ID

    # Create a RunnableConfig instance
config = RunnableConfig(
    configurable={
        "session_id": session_id,
        "thread_id": thread_id
        }
    )

    # Initialize the StreamlitCallbackHandler
st_callback = StreamlitCallbackHandler(st.container())

    # Initialize a variable to hold the final output content
final_output_content = ""

    # Prepare the input for the invoke method
input_data = {
    "messages": messages,
    "input": query # Ensure the input key is included
}

# Invoke the agent with the messages and configuration, including the callback
response = agent_executor.invoke(query, config, callbacks=[st_callback])

    
response

2025-01-01 07:24:30.333 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:24:30.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:24:30.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:24:30.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:24:30.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:24:30.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [9]:
# List the physicians specializing in Pulmonology at Penang General Hospital.
# do Lam Wah Ee Hospital exist in the database? May I know its contact number
query = "do Lam Wah Ee Hospital exist in the database? May I know its contact number"

response = generate_response(query)

response


2025-01-01 07:21:43.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:21:43.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:21:43.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:21:43.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:21:43.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 07:21:43.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


KeyError: 'tool_call_id'